In [1725]:
import pandas as pd

gdp_pc = pd.read_csv("datafiles/gdp_pc.csv", sep = ",", header = 0)
inflation = pd.read_csv("datafiles/inflation.csv", sep = ",", header = 0)
trade_pct = pd.read_csv("datafiles/trade_pct.csv", sep = ",", header = 0)
unemployment = pd.read_csv("datafiles/unemployment.csv", sep = ",", header = 0)
total_export = pd.read_csv("datafiles/total_export.csv", sep = ",", header = 0)
export_neighbors = pd.read_csv("datafiles/export_neighbors.csv", sep = ",", header = 0)
import_neighbors = pd.read_csv("datafiles/import_neighbors.csv", sep = ",", header = 0)
neighbors = pd.read_excel("datafiles/landlocked_neighbors.xlsx")

years = [i for i in range(2014,2019)]

In [1726]:
total_export = pd.melt(total_export, id_vars = 'Country Code', value_vars = total_export.columns[1:], var_name = 'year', value_name = 'total_export')
export_neighbors = pd.melt(export_neighbors, id_vars = 'Country Code', value_vars = export_neighbors.columns[1:], var_name = 'year', value_name = 'export_neighbors')
import_neighbors = pd.melt(import_neighbors, id_vars = 'Country Code', value_vars = import_neighbors.columns[1:], var_name = 'year', value_name = 'import_neighbors')

In [1727]:
trade = pd.merge(total_export, export_neighbors)
trade = pd.merge(trade, import_neighbors)

trade['trade_ratio'] = trade.export_neighbors / trade.import_neighbors
trade['trade_share'] = trade.export_neighbors / trade.total_export
trade.head()

,Country Code,year,total_export,export_neighbors,import_neighbors,trade_ratio,trade_share
0,ARM,2014,1.490190e+06,1.498365e+05,5.085286e+05,0.294647,0.100549
1,AUT,2014,1.697129e+08,8.744547e+07,1.012770e+08,0.863429,0.515255
2,AZE,2014,2.175174e+07,1.711755e+06,2.842167e+06,0.602271,0.078695
3,BDI,2014,1.415007e+05,3.756512e+04,6.850044e+04,0.548392,0.265477
4,BFA,2014,2.845603e+06,6.010152e+05,1.768104e+06,0.339921,0.211208


In [1728]:
inflation = pd.melt(inflation, id_vars = 'Country Code', value_vars = inflation.columns[4:], var_name = 'year', value_name = 'inflation')
trade_pct = pd.melt(trade_pct, id_vars = 'Country Code', value_vars = trade_pct.columns[4:], var_name = 'year', value_name = 'trade_pct')
unemployment = pd.melt(unemployment, id_vars = 'Country Code', value_vars = unemployment.columns[4:], var_name = 'year', value_name = 'unemployment')
gdp_pc = pd.melt(gdp_pc, id_vars = 'Country Code', value_vars = gdp_pc.columns[4:], var_name = 'year', value_name = 'gdp_pc')

In [1729]:
factors = pd.merge(gdp_pc, inflation)
factors = pd.merge(factors, trade_pct)
factors = pd.merge(factors, unemployment)
factors['year'] = factors['year'].str.slice(0, 4)

In [1730]:
factors[factors['Country Code'] == 'LAO']

,Country Code,year,gdp_pc,inflation,trade_pct,unemployment
108,LAO,2014,1999.95820314913,4.12924306873775,99.0597382925996,0.763000011444092
999,LAO,2015,2140.04432280044,1.27735427102986,85.7983217037662,0.777999997138977
1890,LAO,2016,2324.40091700425,1.59691227583204,75.0918909206445,0.791999995708466
2781,LAO,2017,2455.21155944893,0.82547860756813,..,0.805000007152557
3672,LAO,2018,2569.09389217251,2.04039266890198,..,0.827000021934509


In [1731]:
for i in factors.columns[2:]:
    factors[i] = pd.to_numeric(factors[i], errors = 'coerce')

In [1732]:
factors[factors['Country Code'] == 'LAO']

,Country Code,year,gdp_pc,inflation,trade_pct,unemployment
108,LAO,2014,1999.958203,4.129243,99.059738,0.763
999,LAO,2015,2140.044323,1.277354,85.798322,0.778
1890,LAO,2016,2324.400917,1.596912,75.091891,0.792
2781,LAO,2017,2455.211559,0.825479,NaN,0.805
3672,LAO,2018,2569.093892,2.040393,NaN,0.827


In [1733]:
landlocked = pd.merge(trade, factors)
landlocked['Country Code'] = landlocked['Country Code'].astype("category")
landlocked['year'] = landlocked['year'].astype("category")

landlocked = landlocked.sort_values(by=['Country Code','year'])
landlocked = landlocked.drop(['total_export', 'export_neighbors', 'import_neighbors'], axis = 1)


In [1734]:
landlocked.isna().any()

Country Code    False
year            False
trade_ratio     False
trade_share      True
gdp_pc          False
inflation       False
trade_pct        True
unemployment    False
dtype: bool

In [1735]:
landlocked[landlocked['Country Code'] == 'LAO']

,Country Code,year,trade_ratio,trade_share,gdp_pc,inflation,trade_pct,unemployment
15,LAO,2014,0.512666,0.801205,1999.958203,4.129243,99.059738,0.763
45,LAO,2015,0.751664,0.871981,2140.044323,1.277354,85.798322,0.778
75,LAO,2016,0.716890,0.850069,2324.400917,1.596912,75.091891,0.792
105,LAO,2017,0.991781,0.884307,2455.211559,0.825479,NaN,0.805
135,LAO,2018,1.049078,0.901424,2569.093892,2.040393,NaN,0.827


In [1736]:
landlocked = landlocked.fillna(method = "ffill")
landlocked.isna().any()

Country Code    False
year            False
trade_ratio     False
trade_share     False
gdp_pc          False
inflation       False
trade_pct       False
unemployment    False
dtype: bool

In [1737]:
landlocked[landlocked['Country Code'] == 'LAO']

,Country Code,year,trade_ratio,trade_share,gdp_pc,inflation,trade_pct,unemployment
15,LAO,2014,0.512666,0.801205,1999.958203,4.129243,99.059738,0.763
45,LAO,2015,0.751664,0.871981,2140.044323,1.277354,85.798322,0.778
75,LAO,2016,0.716890,0.850069,2324.400917,1.596912,75.091891,0.792
105,LAO,2017,0.991781,0.884307,2455.211559,0.825479,75.091891,0.805
135,LAO,2018,1.049078,0.901424,2569.093892,2.040393,75.091891,0.827


In [1738]:
landlocked.to_csv("datafiles/landlocked.csv")